In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import time

In [166]:


df = pd.read_csv('tubedata.csv', 
                 names=['start','end','tube_line','average_time','zone1','zone2'],
                header=None)
df


,start,end,tube_line,average_time,zone1,zone2
0,Harrow & Wealdstone,Kenton,Bakerloo,3,5,0
1,Kenton,South Kenton,Bakerloo,2,4,0
2,South Kenton,North Wembley,Bakerloo,2,4,0
3,North Wembley,Wembley Central,Bakerloo,2,4,0
4,Wembley Central,Stonebridge Park,Bakerloo,3,4,0
...,...,...,...,...,...,...
369,Victoria,Pimlico,Victoria,3,1,0
370,Pimlico,Vauxhall,Victoria,1,1,0
371,Vauxhall,Stockwell,Victoria,3,1,2
372,Stockwell,Brixton,Victoria,2,2,0


In [184]:

# Get all rows with Paddington and Edgware Road
    
df[(df['start']=='Paddington') & (df['end']=='Edgware Road')]

,start,end,tube_line,average_time,zone1,zone2
13,Paddington,Edgware Road,Bakerloo,3,1,0
73,Paddington,Edgware Road,Circle,3,1,0
176,Paddington,Edgware Road,Hammersmith & City,4,1,0


In [186]:
def step_dictionary(df):
    station_dict = {}
    zone_dict = {}
    tube_line_dict = {}

    # get data row by row
    for index, row in df.iterrows():
    
        start_station = row[0]
        end_station = row[1]
        tube_line = row[2]
        act_cost = int(row[3]) 

        zone1 = row[4]
        zone2 = row[5]


        # station dictionary of child station tuples (child_name, cost from parent to the child)
        # {"Mile End": [("Stepney Green", 2), ("Wembley", 1)]}
        
        # Add entry for start_station if not present
        if start_station not in station_dict:
            station_dict[start_station] = []
        
        # Add entry for end_station if not present
        if end_station not in station_dict:
            station_dict[end_station] = []

        station_dict[start_station].append((end_station, act_cost, tube_line))
        station_dict[end_station].append((start_station, act_cost, tube_line))  # add the other direction of the tube "step"

        # add the main zone
        if start_station not in zone_dict:
            zone_dict[start_station] = set()
        zone_dict[start_station].add(zone1)

        # add the secondary zone
        if end_station not in zone_dict:
            zone_dict[end_station] = set()

        if zone2 != "0":
            zone_dict[start_station].add(zone2)
            # if the secondary zone is not 0 it's the main zone for the ending station
            zone_dict[end_station].add(zone2)
        else:
            # otherwise the main zone for the ending station is the same as for the starting station
            zone_dict[end_station].add(zone1)

        if start_station not in tube_line_dict:
            tube_line_dict[start_station] = set()
        if end_station not in tube_line_dict:
            tube_line_dict[end_station] = set()
            
        tube_line_dict[start_station].add(tube_line)
        tube_line_dict[end_station].add(tube_line)

    return station_dict, zone_dict, tube_line_dict

In [189]:
station_dict['Paddington']

[('Warwick Avenue', 2, 'Bakerloo'),
 ('Edgware Road', 3, 'Bakerloo'),
 ('Edgware Road', 3, 'Circle'),
 ('Bayswater', 2, 'Circle'),
 ('Edgware Road', 3, 'District'),
 ('Bayswater', 2, 'District'),
 ('Royal Oak', 1, 'Hammersmith & City'),
 ('Edgware Road', 4, 'Hammersmith & City')]

In [251]:
station_dict, zone_dict, tube_line_dict = step_dictionary(df)
station_names = list(station_dict.keys())

inv_station_dict = {}
for i, station in enumerate(station_names):
    inv_station_dict[station] = i



# Use lookup dictionary to create adjacency matrix; graphs can have multiple edges between nodes

def create_adjacency_matrix(station_dict):
    n = len(station_dict)
    adj_matrix = [[np.inf for _ in range(n)] for _ in range(n)]
    for station in station_dict:
        for child, cost, _ in station_dict[station]:
            # Assumption: if there are multiple edges between two nodes, the cost is the minimum cost
            if cost < adj_matrix[station_names.index(station)][station_names.index(child)]:
                adj_matrix[station_names.index(station)][station_names.index(child)]= cost
    return adj_matrix

adj_matrix = create_adjacency_matrix(station_dict)


def inverse_zone_dict(zone_dict):
    inverse_zone_dict = {}
    for station in zone_dict:
        for zone in zone_dict[station]:
            if zone not in inverse_zone_dict:
                inverse_zone_dict[zone] = set()
            inverse_zone_dict[zone].add(station)
    return inverse_zone_dict

inverse_zone_dict(zone_dict)

{'5': {'Barkingside',
  'Becontree',
  'Buckhurst Hill',
  'Burnt Oak',
  'Canons Park',
  'Chigwell',
  'Cockfosters',
  'Dagenham East',
  'Dagenham Heathway',
  'Eastcote',
  'Edgware',
  'Elm Park',
  'Fairlop',
  'Gants Hill',
  'Grange Hill',
  'Greenford',
  'Hainault',
  'Harrow & Wealdstone',
  'Harrow-on-the-Hill',
  'Hatton Cross',
  'High Barnet',
  'Hounslow West',
  'Kenton',
  'Loughton',
  'North Harrow',
  'Northolt',
  'Northwood Hills',
  'Oakwood',
  'Pinner',
  'Queensbury',
  'Rayners Lane',
  'Roding Valley',
  'Ruislip Gardens',
  'Ruislip Manor',
  'South Harrow',
  'South Ruislip',
  'Southgate',
  'Stanmore',
  'Totteridge & Whetstone',
  'West Harrow',
  'Woodford'},
 '4': {'Alperton',
  'Arnos Grove',
  'Barking',
  'Barkingside',
  'Becontree',
  'Boston Manor',
  'Bounds Green',
  'Brent Cross',
  'Buckhurst Hill',
  'Burnt Oak',
  'Colindale',
  'East Finchley',
  'East Ham',
  'Finchley Central',
  'Gants Hill',
  'Greenford',
  'Gunnersbury',
  'Hanger

In [250]:
station_names.index('Paddington')   == inv_station_dict['Paddington']

True

In [259]:
class TubeStationGraph:
    def __init__(self, df):
        self.station_dict, self.zone_dict, self.tube_line_dict = step_dictionary(df)    # station_dict ----> {station: [(child_station, cost, tube_line)]}
                                                                                        # zone_dict ------> {station: {zone1, zone2}}
                                                                                        # tube_line_dict -----> {station: {tube_line1, tube_line2, ...}}
        self.station_names = list(self.station_dict.keys())                             # list of all station names
        self.adj_matrix = self.create_adjacency_matrix()                                # adjacency matrix for cost between stations
        self.inverse_zone_dict = self.inverse_zone_dict()                               # inverse_zone_dict ----> {zone: {station1, station2, ...}}
        

    def create_adjacency_matrix(self):
        n = len(self.station_dict)
        adj_matrix = [[np.inf for _ in range(n)] for _ in range(n)]
        for station in self.station_dict:
            for child, cost, _ in self.station_dict[station]:
                # Assumption: if there are multiple edges between two nodes, the cost is the minimum cost
                if cost < adj_matrix[self.station_names.index(station)][self.station_names.index(child)]:
                    adj_matrix[self.station_names.index(station)][self.station_names.index(child)]= cost
        return adj_matrix
    
    def inverse_zone_dict(self):
        inverse_zone_dict = {}
        for station in self.zone_dict:
            for zone in self.zone_dict[station]:
                if zone not in inverse_zone_dict:
                    inverse_zone_dict[zone] = set()
                inverse_zone_dict[zone].add(station)
        return inverse_zone_dict
    

# class SearchProblem:
#     def __init__(self, graph, start, goal):
#         self.graph = graph
#         self.start = start
#         self.goal = goal

#         self.state = (self.start, [start], )

In [ ]:
G = nx.from_numpy_matrix(np.array(adj_matrix))
plt.figure(figsize=(30,30))
labels = {i: station_names[i] for i in range(n)}
nx.draw(G, with_labels=True, labels=labels)
plt.show()

In [260]:
tube_graph = TubeStationGraph(df)

In [262]:
# Breadth First Search

def bfs(start, end, tube_graph):
    '''
    Breadth First Search

    Parameters:
    ------------
    start: str
        starting station
    end: str
        ending station
    tube_graph: instance of class TubeStationGraph
        provides adjacency matrix and lookup dictionaries


    Returns:
    ------------
    path: list
        list of stations in the path
    time_taken: float
        total average time to traverse the path
    expanded_nodes: int
        number of nodes expanded 

    '''
    # Enqueue state = (node, path traversed, cost)
    queue = [(start, [start], 0)]
    visited = set()
    while queue:
        # Dequeue
        (node, path, cost) = queue.pop(0)
        if node not in visited:
            if node == end:    
            # Goal Test successful
                time_taken = cost           
                expanded_nodes = len(visited)
                return path, time_taken, expanded_nodes
            visited.add(node)

            # Expand node using the adjacency matrix
            for child,  child_cost in enumerate(tube_graph.adj_matrix[tube_graph.station_names.index(node)]):
                if child_cost != np.inf:        # child_cost = np.inf indicates no connection
                    queue.append((tube_graph.station_names[child], path + [tube_graph.station_names[child]], cost + child_cost))

    return [], len(visited)          # Retun failure


# Depth First Search

def dfs(start, end, tube_graph):
    '''
    Depth First Search

    Parameters:
    ------------
    start: str
        starting station
    end: str
        ending station
    tube_graph: instance of class TubeStationGraph
        provides adjacency matrix and lookup dictionaries


    Returns:
    ------------
    path: list
        list of stations in the path
    time_taken: float
        total average time to traverse the path
    expanded_nodes: int
        number of nodes expanded 

    '''
    # Push state = (node, path traversed, cost)
    stack = [(start, [start], 0)]              
    visited = set()
    while stack:
        # Pop state
        (node, path, cost) = stack.pop()
        if node not in visited:       
            if node == end:    
            # Goal Test successful
                time_taken = cost           
                expanded_nodes = len(visited)
                return path, time_taken, expanded_nodes
            visited.add(node)

            # Expand node
            for child,  child_cost in enumerate(tube_graph.adj_matrix[tube_graph.station_names.index(node)]):
                if child_cost != np.inf:        # child_cost = np.inf indicates no connection
                    stack.append((tube_graph.station_names[child], path + [tube_graph.station_names[child]], cost + child_cost))

    return [], len(visited)          # Retun failure


# Uniform Cost Search  
def ucs(start, end, tube_graph):
    '''
    Uniform Cost Search

    Parameters:
    ------------
    start: str
        starting station
    end: str
        ending station
    tube_graph: instance of class TubeStationGraph
        provides adjacency matrix and lookup dictionaries


    Returns:
    ------------
    path: list
        list of stations in the path
    time_taken: float
        total average time to traverse the path
    expanded_nodes: int
        number of nodes expanded 

    '''

    # Enqueue state = (node, path traversed, cost)
    queue = [(start, [start], 0)]          
    visited = set()
    while queue:
        # Dequeue state
        (node, path, cost) = queue.pop(0)
        if node not in visited:
            if node == end:    
            # Goal Test successful
                time_taken = cost           
                expanded_nodes = len(visited)
                return path, time_taken, expanded_nodes
            visited.add(node)

            # Expand node
            for child,  child_cost in enumerate(tube_graph.adj_matrix[tube_graph.station_names.index(node)]):
                if child_cost != np.inf:        # child_cost = np.inf indicates no connection
                    queue.append((tube_graph.station_names[child], path + [tube_graph.station_names[child]], cost + child_cost))
            queue.sort(key=lambda x: x[-1])          # Sort queue by cost

    return [], -1, len(visited)          # Retun failure

In [263]:
path, t, space = bfs('Mile End', 'New Cross Gate', tube_graph)

print(f"Path: {path}")
print(f"Total cost: {t}")
print(f"Total nodes visited: {space}")

total_cost(path, adj_matrix)

Path: ['Mile End', 'Bethnal Green', 'Liverpool Street', 'Bank/Monument', 'London Bridge', 'Bermondsey', 'Canada Water', 'Surrey Quays', 'New Cross Gate']
Total cost: 20
Total nodes visited: 90
 Mile End to Bethnal Green costs 2
 Bethnal Green to Liverpool Street costs 3
 Liverpool Street to Bank/Monument costs 2
 Bank/Monument to London Bridge costs 2
 London Bridge to Bermondsey costs 3
 Bermondsey to Canada Water costs 2
 Canada Water to Surrey Quays costs 2
 Surrey Quays to New Cross Gate costs 4


20

In [265]:
path, t, space = dfs('Mile End', 'New Cross Gate', tube_graph)

print(f"Path: {path}")
print(f"Total cost: {t}")
print(f"Total nodes visited: {space}")

total_cost(path, adj_matrix)

Path: ['Mile End', 'Bow Road', 'Bromley-by-Bow', 'West Ham', 'Canning Town', 'North Greenwich', 'Canary Wharf', 'Canada Water', 'Surrey Quays', 'New Cross Gate']
Total cost: 23
Total nodes visited: 237
 Mile End to Bow Road costs 1
 Bow Road to Bromley-by-Bow costs 2
 Bromley-by-Bow to West Ham costs 2
 West Ham to Canning Town costs 3
 Canning Town to North Greenwich costs 3
 North Greenwich to Canary Wharf costs 3
 Canary Wharf to Canada Water costs 3
 Canada Water to Surrey Quays costs 2
 Surrey Quays to New Cross Gate costs 4


23

In [266]:
path, t, space = ucs('Mile End', 'New Cross Gate', tube_graph)

print(f"Path: {path}")
print(f"Total cost: {t}")
print(f"Total nodes visited: {space}")

total_cost(path, adj_matrix)

Path: ['Mile End', 'Stepney Green', 'Whitechapel', 'Shadwell', 'Wapping', 'Rotherhithe', 'Canada Water', 'Surrey Quays', 'New Cross Gate']
Total cost: 16
Total nodes visited: 66
 Mile End to Stepney Green costs 2
 Stepney Green to Whitechapel costs 3
 Whitechapel to Shadwell costs 2
 Shadwell to Wapping costs 1
 Wapping to Rotherhithe costs 1
 Rotherhithe to Canada Water costs 1
 Canada Water to Surrey Quays costs 2
 Surrey Quays to New Cross Gate costs 4


16

In [177]:
# Improve and implement the current UCS cost function to include the time to change lines at one station (e.g., 2 minutes).

def ucs_improved_old(start, end, inv_station_dict, adj_matrix, tube_line_dict, change_time=2):
    '''
    Uniform Cost Search

    Parameters:
    ------------
    start: str
        starting station
    end: str
        ending station
    inv_station_dict: dict
        lookup dictionary for station names
    adj_matrix: list
        adjacency matrix for the graph

    Returns:
    ------------
    path: list
        list of stations in the path

    time: float
        time taken to run the algorithm

    visited: int
        number of nodes visited

    '''

    lines = tube_line_dict[start]
    queue = [(start, [start], 0, lines)]           # state = (node, path traversed, cost, available_lines)
    visited = set()
    while queue:
        # Get node from queue
        (node, path, cost, lines) = queue.pop(0)
        if node not in visited:
            if node == end:    
            # Goal Test successful
                time_taken = cost           
                expanded_nodes = len(visited)
                return path, time_taken, expanded_nodes
            visited.add(node)
        
            # Expand node
            for child, child_cost in enumerate(adj_matrix[inv_station_dict[node]]):
                if child_cost != 0:         # child_cost = 0 indicates no connection
                    child_lines = tube_line_dict[station_names[child]]
                    if len(set(lines).intersection(child_lines)) > 0:
                        queue.append((station_names[child], path + [station_names[child]], cost + child_cost, child_lines))
                    else:
                        queue.append((station_names[child], path + [station_names[child]], cost + child_cost + change_time, child_lines))
            queue.sort(key=lambda x: x[-2])          # Sort queue by cost

    
    return [], -1, len(visited)          # Retun failure

In [315]:
def ucs_improved(start, end, tube_graph, change_time=2):
    '''
    Uniform Cost Search with improved cost function

    Parameters:
    ------------
    start: str
        starting station
    end: str
        ending station
    tube_graph: instance of class TubeStationGraph
        provides adjacency matrix and lookup dictionaries
    change_time: int
        time taken to change lines

    Returns:
    ------------
    path: list
        list of stations in the path

    time: float
        time taken to run the algorithm

    visited: int
        number of nodes visited

    '''
    # The state requires a modification to include th last line used
    queue = [(start, [start], 0, None)]        # state = (node, path traversed, cost, last line used)
    visited = set()
    while queue:
        # Get node from queue
        (node, path, cost, last_line_type) = queue.pop(0)
        if node not in visited:
            if node == end:    
            # Goal Test successful
                time_taken = cost           
                expanded_nodes = len(visited)
                return path, time_taken, expanded_nodes
            visited.add(node)
        
            # Expand node
            for child, child_cost, line in tube_graph.station_dict[node]:
                line_change_cost = 0 if last_line_type is None or last_line_type == line else change_time
                queue.append((child, path + [child], cost + child_cost + line_change_cost, line))
            queue.sort(key=lambda x: x[-2])          # Sort queue by cost

        
    return [], -1, len(visited)          # Retun failure

In [322]:
path, t, space = ucs_improved('Hammersmith', 'Heathrow Terminal 4', tube_graph)

print(f"Path: {path}")
print(f"Total cost: {t}")
print(f"Total nodes visited: {space}")

total_cost(path, adj_matrix)

Path: ['Hammersmith', 'Turnham Green', 'Acton Town', 'South Ealing', 'Northfields', 'Boston Manor', 'Osterley', 'Hounslow East', 'Hounslow Central', 'Hounslow West', 'Hatton Cross', 'Heathrow Terminals 1-2-3', 'Heathrow Terminal 4']
Total cost: 34
Total nodes visited: 168
 Hammersmith to Turnham Green costs 3
 Turnham Green to Acton Town costs 3
 Acton Town to South Ealing costs 4
 South Ealing to Northfields costs 1
 Northfields to Boston Manor costs 2
 Boston Manor to Osterley costs 3
 Osterley to Hounslow East costs 2
 Hounslow East to Hounslow Central costs 2
 Hounslow Central to Hounslow West costs 2
 Hounslow West to Hatton Cross costs 4
 Hatton Cross to Heathrow Terminals 1-2-3 costs 3
 Heathrow Terminals 1-2-3 to Heathrow Terminal 4 costs 5
Total cost without line changing penalty: 34


In [269]:
# Calculate total cost of the path
def total_cost(path, adj_matrix):

    total_cost = 0
    for i in range(len(path)-1):
        total_cost += adj_matrix[inv_station_dict[path[i]]][inv_station_dict[path[i+1]]]
        print(f" {path[i]} to {path[i+1]} costs {adj_matrix[inv_station_dict[path[i]]][inv_station_dict[path[i+1]]]}")
    print(f"Total cost without line changing penalty: {total_cost}")



In [275]:
def zone_distance(tube_graph, n_samples=10, seed=42):
    '''
    Calculate distance between zones by sampling 10 random stations from each zone and doing breadth first search between them
    '''
    np.random.seed(seed)
    zone_names = list(tube_graph.inverse_zone_dict.keys())
    zone_distance_matrix = [[0 for _ in range(len(zone_names))] for _ in range(len(zone_names))]
    for i in range(len(zone_names)):
        for j in range(i+1, len(zone_names)):
            total_distance = 0
            for _ in range(n_samples):
                start = np.random.choice(list(tube_graph.inverse_zone_dict[zone_names[i]]))
                end = np.random.choice(list(tube_graph.inverse_zone_dict[zone_names[j]]))
                path, _, _ = bfs(start, end, tube_graph)
                total_distance += len(path)
            zone_distance_matrix[i][j] = total_distance / n_samples
            zone_distance_matrix[j][i] = total_distance / n_samples

    return zone_distance_matrix, zone_names

zone_distance_matrix, zone_names = zone_distance(tube_graph)
print(f"Zone list: {zone_names}")
zone_distance_matrix

Zone list: ['5', '4', '3', '2', '1', '6', 'a', 'b', 'c', 'd']


[[0, 21.8, 20.3, 18.5, 15.5, 21.2, 18.7, 21.0, 24.0, 24.9],
 [21.8, 0, 16.9, 13.2, 14.7, 19.6, 22.5, 23.8, 23.9, 24.9],
 [20.3, 16.9, 0, 12.7, 11.5, 19.4, 20.9, 21.7, 24.0, 23.2],
 [18.5, 13.2, 12.7, 0, 7.9, 20.5, 17.1, 19.8, 20.1, 20.7],
 [15.5, 14.7, 11.5, 7.9, 0, 16.4, 16.1, 17.5, 19.5, 19.5],
 [21.2, 19.6, 19.4, 20.5, 16.4, 0, 24.6, 24.7, 29.7, 26.1],
 [18.7, 22.5, 20.9, 17.1, 16.1, 24.6, 0, 3.2, 4.1, 5.1],
 [21.0, 23.8, 21.7, 19.8, 17.5, 24.7, 3.2, 0, 2.0, 3.2],
 [24.0, 23.9, 24.0, 20.1, 19.5, 29.7, 4.1, 2.0, 0, 1.8],
 [24.9, 24.9, 23.2, 20.7, 19.5, 26.1, 5.1, 3.2, 1.8, 0]]

In [303]:
# Best-first Search using zone distance heuristic

def heuristic_fn(node, end, tube_graph, zone_distance_matrix, zone_names):
    node_zone_set = tube_graph.zone_dict[node]
    end_zone_set = tube_graph.zone_dict[end]
    # Get the zone with the minimum distance from the end zone
    if node_zone_set.intersection(end_zone_set):
        return 0
    else:
        d = np.inf
        for e in end_zone_set:
            for n in node_zone_set:
                if d > zone_distance_matrix[zone_names.index(n)][zone_names.index(e)]: 
                    d = zone_distance_matrix[zone_names.index(n)][zone_names.index(e)]
        
    return d



def best_first_search(start, end, tube_graph, zone_distance_matrix, zone_names, change_time=2):
    '''
    Best First Search

    Parameters:
    ------------
    start: str
        starting station
    end: str
        ending station
    tube_graph: instance of class TubeStationGraph
        provides adjacency matrix and lookup dictionaries
    zone_distance_matrix: list
        matrix of distances between zones
    zone_names: list
        list of zone names
    
    Returns:
    ------------
    path: list
        list of stations in the path
    time_taken: float
        total average time to traverse the path
    expanded_nodes: int
        number of nodes expanded
    '''

    # Enqueue state = (node, path traversed, cost, last line used, heuristic)
    queue = [(start, [start], 0, None, 0)]
    visited = set()
    while queue:
        (node, path, cost, last_line_type, heuristic) = queue.pop(0)
        if node not in visited:
            if node == end:    
            # Goal Test successful
                time_taken = cost           
                expanded_nodes = len(visited)
                return path, time_taken, expanded_nodes
            visited.add(node)

            # for child, child_cost in enumerate(tube_graph.adj_matrix[tube_graph.station_names.index(node)]):
            #     if child_cost != np.inf:
            #         # Calculate heuristic for child node
            #         heuristic = heuristic_fn(station_names[child], end, tube_graph, zone_distance_matrix, zone_names)
            #         queue.append((station_names[child], path + [station_names[child]], cost + child_cost, None, heuristic))
            # queue.sort(key=lambda x: x[-1])          # Sort queue by heuristic

            # Expand node
            for child, child_cost, line in tube_graph.station_dict[node]:
                line_change_cost = 0 if last_line_type is None or last_line_type == line else change_time
                heuristic = heuristic_fn(child, end, tube_graph, zone_distance_matrix, zone_names)
                queue.append((child, path + [child], cost + child_cost + line_change_cost, line, heuristic))
            queue.sort(key=lambda x: x[-1])          # Sort queue by heuristic

    return [], len(visited)          # Retun failure

In [321]:
path, t, space = best_first_search('Hammersmith', 'Heathrow Terminal 4', tube_graph, zone_distance_matrix, zone_names)

print(f"Path: {path}")
print(f"Total cost: {t}")
print(f"Total nodes visited: {space}")

total_cost(path, adj_matrix)

Path: ['Hammersmith', 'Turnham Green', 'Acton Town', 'South Ealing', 'Northfields', 'Boston Manor', 'Osterley', 'Hounslow East', 'Hounslow Central', 'Hounslow West', 'Hatton Cross', 'Heathrow Terminals 1-2-3', 'Heathrow Terminal 4']
Total cost: 34
Total nodes visited: 32
 Hammersmith to Turnham Green costs 3
 Turnham Green to Acton Town costs 3
 Acton Town to South Ealing costs 4
 South Ealing to Northfields costs 1
 Northfields to Boston Manor costs 2
 Boston Manor to Osterley costs 3
 Osterley to Hounslow East costs 2
 Hounslow East to Hounslow Central costs 2
 Hounslow Central to Hounslow West costs 2
 Hounslow West to Hatton Cross costs 4
 Hatton Cross to Heathrow Terminals 1-2-3 costs 3
 Heathrow Terminals 1-2-3 to Heathrow Terminal 4 costs 5
Total cost without line changing penalty: 34
